In [2]:
import pandas as pd
import json
from collections import defaultdict
import re
import numpy as np
from utilities import Tiempo
import dataExtraction
import statistics
import keyword

import xml.etree.ElementTree as ET

In [ ]:
#Si tenerEnCuentaMove = False -> cuando se realiza una accion de tipo "move" no se tiene en cuenta para calcular el tiempo entre interacciones, como si no existiese la traza
def extraerTiemposEntreInteracciones(rawData, tenerEnCuentaMove = False):
    ultimaInteraccionJugador = defaultdict()
    
    tiempoInicio_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre inicio de nivel y la siguiente interaccion (reinicio, completado o interaccion con tarjeta)
    tiempoInteraccion_Completado_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion (inicio, reinicio o interaccion con tarjeta) y terminar el nivel
    tiempoInteraccion_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion con tarjeta y otra interaccion con tarjeta
    
    anomalias = []
    
    erInteracted = re.compile(r'\binteracted$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erLevelExitButton = re.compile(r'\blevel_exit_button$\b') 
    erInitialized = re.compile(r'\binitialized$\b')
    erSeriousGame = re.compile(r'\bserious-game$\b')    
    erCompleted = re.compile(r'\bcompleted$\b')
    erLevel = re.compile(r'\blevel$\b')

        
    for e in rawData:
        verb = e["verb"]["id"]
        obj = e["object"]["definition"]["type"]
        objectId = e["object"]["id"]
        name = e["actor"]["name"]
        timestamp = e["timestamp"]
        
        erIdLevel = re.compile(r'/')
        levelCode = erIdLevel.split(objectId)[-1]
        
        if(levelCode != "editor_level"):
            
            if erSeriousGame.search(obj) and erInitialized.search(verb): #Si inicia el juego se reinicia su ultima interaccion
                if name in ultimaInteraccionJugador:
                    del ultimaInteraccionJugador[name]

            elif erLevel.search(obj):
                if erInitialized.search(verb): #Significa que ha iniciado o reiniciado el nivel
                    if "result" in e: #Inicia el nivel desde el menu
                        ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "inicio"}
                    else: #Reinicia el nivel
                        try:
                            t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                            
                            if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                                if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "reinicio"})
                                else:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "reinicio"}]
                            #Actualizamos ultima interaccion
                            ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "reinicio"}
                        except:
                            #Si entra aqui es porque ha reiniciado un nivel que no habia iniciado
                            #Trazas estan mal
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha reiniciado un nivel que no estaba iniciado"})

                elif erCompleted.search(verb) and e["result"]["score"]["raw"] != 0:
                    #Si completa un nivel o sale al menu se reinicia su ultima interaccion, si falla el intento pero vuelve a intentarlo no se reinicia
                    #Si raw == 0 significa que se reinicia el nivel
                    try:
                        t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)                        
                        #Añadimos la dif de tiempo al diccionario del jugador
                        if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                            if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "terminado", "exito" : e["result"]["success"]})
                            else:
                                tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "terminado", "exito" : e["result"]["success"]}]
                                
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "inicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "inicio"}]
                                    
                        elif ultimaInteraccionJugador[name]["accion"] == "reinicio":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "reinicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "reinicio"}]

                        elif ultimaInteraccionJugador[name]["accion"] == "interaccion":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "interaccionTarjeta"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "interaccionTarjeta"}]
                        #Actualizamos ultima interaccion, la borramos
                        del ultimaInteraccionJugador[name]
                    except:
                        #Si entra aqui es porque se ha completado un nivel que no se habia iniciado
                        #Probablemente por anomalias en las trazas
                        anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo": "Se ha completado un nivel que no estaba iniciado"})

            elif erGameObject.search(obj) and erInteracted.search(verb):
                if "result" in e and "extensions" in e["result"] and "level" in e["result"]["extensions"]:
                    levelCode = e["result"]["extensions"]["level"]
                    if not erLevelExitButton.search(objectId) and levelCode != "editor_level":
                        if name in ultimaInteraccionJugador:
                            if e["result"]["extensions"]["action"] != "move" or tenerEnCuentaMove: #<-- Que accion realiza
                                t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                                #Añadimos la dif de tiempo al diccionario del jugador

                                if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                    if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                        tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "interaccionTarjeta"})
                                    else:
                                        tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "interaccionTarjeta"}]

                                if ultimaInteraccionJugador[name]["accion"] == "interaccionTarjeta":
                                    if levelCode in tiempoInteraccion_Interaccion_Jugador[name]:
                                        tiempoInteraccion_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "interaccionTarjeta"})
                                    else:
                                        tiempoInteraccion_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "interaccionTarjeta"}]

                                ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "interaccionTarjeta"}
                        else:
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha interactuado en un nivel que no estaba iniciado"})
                        
                    
    return {"tiempoInicio_Interaccion_Jugador" : tiempoInicio_Interaccion_Jugador, "tiempoInteraccion_Completado_Jugador" : tiempoInteraccion_Completado_Jugador, "tiempoInteraccion_Interaccion_Jugador" : tiempoInteraccion_Interaccion_Jugador,"anomalias" : anomalias}

In [ ]:
def extraerTiemposPorNivelJugador(rawData):
    
    tiempos = defaultdict(defaultdict)
    intentosNecesarios = defaultdict(defaultdict)
    
    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    
    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]
        
        if erLevel.search(obj): #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if levelCode != "editor_level":
                if erInitialized.search(verb): #Si la acción es inicio o reinicio
                    if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode].append({"intentos" : 1, "success" : False})
                            tiempos[name][levelCode].append({"ini" : timestamp, "fin" : None, "stars" : ""})
                        else:
                            intentosNecesarios[name][levelCode] = [{"intentos" : 1, "success" : False}]
                            tiempos[name][levelCode] = [{"ini" : timestamp, "fin" : None, "stars" : ""}]
                    else:
                        intentosNecesarios[name][levelCode][-1]["intentos"] += 1
                elif erCompleted.search(verb):
                    if evento["result"]["score"]["raw"] > 0 :
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode][-1]["success"] = True
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]

                    elif evento["result"]["score"]["raw"] == -1:
                        if levelCode in tiempos[name]:
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
    
    return {"tiempos" : tiempos, "intentosNecesarios" : intentosNecesarios}

In [ ]:
def tiempoPorNiveles_Jugador(data):
    tiemposJugados = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    timeDifference = Tiempo(times["ini"], times["fin"])
                    if level in tiemposJugados[player]:
                        tiemposJugados[player][level].append({"time" : timeDifference, "stars" : times["stars"]})
                    else:
                        tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
    return tiemposJugados

In [ ]:
#TAMBIEN DEVUELVE EL TIEMPO INDIVIDUAL DE CADA JUGADOR PARA COMPLETAR CADA NIVEL
def getMediaTiempoPorNivel(tiempos, soloPrimerExito = True, tiemposOrdenados = False):
    medias = defaultdict(list)
    mediasEstrellas = defaultdict(list)
    tiempoCompletarNivelIndividual = defaultdict(defaultdict)
    
    for player in tiempos:
        for level in tiempos[player]:
            tAux = Tiempo("0s")
            for t in tiempos[player][level]:
                if t["stars"] != -1:
                    if level in medias:
                        medias[level].append(int(tAux + t["time"]))
                        mediasEstrellas[level].append(int(t["stars"]))
                    else:
                        medias[level] = [int(tAux + t["time"])]
                        mediasEstrellas[level] = [int(t["stars"])]
                        
                    tiempoCompletarNivelIndividual[player][level] = tAux + t["time"]
                    if soloPrimerExito:
                        break
                else:
                    tAux += t["time"]
    for m in medias:
        medias[m] = Tiempo(str(int(round(statistics.mean(medias[m]), 0))) + "s")
        
    for m in mediasEstrellas:
        mediasEstrellas[m] = statistics.mean(mediasEstrellas[m])
    
    listaNiveles = medias.keys()
    
    if tiemposOrdenados:
        medias = sorted(medias.items(), key=lambda x: x[1])
        mediasEstrellas = sorted(mediasEstrellas.items(), key=lambda x: x[1])
    else:
        medias = list(medias.items())
        mediasEstrellas = list(mediasEstrellas.items())
    
    return {"mediaTiempos" : medias, "mediaEstrellas" : mediasEstrellas, "listaNiveles" : listaNiveles, "tiemposIndividuales" : tiempoCompletarNivelIndividual}

In [ ]:
def getUltimoNivelAlcanzado(tiempos):
    ultNivel = defaultdict()
    for player in tiempos:
        for level in tiempos[player]:
            for t in tiempos[player][level]:
                ultNivel[player] = level
    return ultNivel

In [ ]:
#TAMBIEN DEVUELVE LOS INTENTOS INDIVIDUALES DE CADA JUGADOR PARA COMPLETAR CADA NIVEL
def getIntentosMedios_HastaCompletarNivel(intentosNecesarios, intentosOrdenados = False):
    intentosMedios = defaultdict(list)
    intentosCompletarNivelIndividual = defaultdict(defaultdict)
    
    for name in intentosNecesarios:
        for level in intentosNecesarios[name]:
            cont = 0
            for i in intentosNecesarios[name][level]:
                cont += i["intentos"]
                if i["success"] == True:
                    intentosMedios[level].append(cont)
                    intentosCompletarNivelIndividual[name][level] = cont
                    break
            
    for level in intentosMedios:
        intentosMedios[level] = round(statistics.mean(intentosMedios[level]), 2)
    
    if intentosOrdenados:
        intentosMedios = sorted(intentosMedios.items(), key=lambda x: x[1])
    else:
        intentosMedios = list(intentosMedios.items())
    return {"intentosMedios" : intentosMedios, "intentosIndividual" : intentosCompletarNivelIndividual}

In [ ]:
def getCategoriasPeorEntendidas(mediaTiempos, mediaEstrellas, intentosMedios, ultNivelCompletado, niveles):
    maxPuntosUltCompletado = 4
    maxPuntosIntentosMedios = 3
    maxPuntosTiempoNivel = 2
    maxPuntosEstrellasNivel = 1
    
    cuantosHanLlegadoAlNivel = defaultdict()
    puntosDificultad = defaultdict()
    
    ultNivelCopia = ultNivelCompletado.copy()
    nJugadores = len(ultNivelCompletado)

    #Contamos cuanta gente ha llegado hasta cada nivel
    for nivel in niveles:
        cuantosHanLlegadoAlNivel[nivel] = len(ultNivelCopia)
        
        keys_to_delete = [k for k, v in ultNivelCopia.items() if v == nivel]
        for k in keys_to_delete:
            del ultNivelCopia[k]
    
    for level in cuantosHanLlegadoAlNivel:
        puntosDificultad[level] = ((nJugadores - cuantosHanLlegadoAlNivel[level]) * maxPuntosUltCompletado)/nJugadores
    
    #Este valor servirá de referencia para obtener los puntos maximos de intentos de un nivel
    maximosIntentosMedios = intentosMedios[-1][1]    
    for i in intentosMedios:
        puntosDificultad[i[0]] += (i[1]*maxPuntosIntentosMedios)/maximosIntentosMedios
        
        
    maximosTiemposMedios = int(mediaTiempos[-1][1])    
    for t in mediaTiempos:
        puntosDificultad[t[0]] += (int(t[1])*maxPuntosTiempoNivel)/maximosTiemposMedios
        
        
    maximosEstrellasMedias = int(mediaEstrellas[-1][1])    
    for e in mediaEstrellas:
        puntosDificultad[e[0]] += (int(e[1])*maxPuntosEstrellasNivel)/maximosEstrellasMedias
    
    puntosDificultad = defaultdict(float, {k: round(v, 2) for k, v in puntosDificultad.items()})
    puntosDificultad = dict(sorted(puntosDificultad.items(), key=lambda x: x[1], reverse=True))
    
    return puntosDificultad

In [ ]:
def get_Tiempos_Ordenados(tiemposInteracciones):
    tiempos = defaultdict()
    
    for name in tiemposInteracciones:
        for level in tiemposInteracciones[name]:
            for t in tiemposInteracciones[name][level]:
                if level in tiempos:
                    tiempos[level].append(t)
                else:
                    tiempos[level] = [t]
                    
    for t in tiempos:
        tiempos[t].sort(key=lambda traza: traza["tiempo"])
        
    return tiempos

In [ ]:
#Devuelve un diccionario con clave nivel y valor el tiempo a partir del cual se considera outlier
def getLimiteOutliers(tiemposNivel):
    limiteOutliers = defaultdict()
    
    for level in tiemposNivel:
        valores = [x["tiempo"] for x in tiemposNivel[level]] #Pasamos los timestamps a un array para calcular los cuartiles
        q1 = np.quantile(valores, 0.25) #Primer cuartil
        q3 = np.quantile(valores, 0.75) #Tercer cuartil
        limiteOutliers[level] = ((q3-q1)*1.5)+q3 #Tiempo donde empieza a considerarse outlier

    return limiteOutliers

In [ ]:
def getJugadoresOutliers(limiteOutliers, tiemposInteracciones):
    outliersPorNivel = defaultdict(defaultdict)
    
    for name in tiemposInteracciones:
        for level in tiemposInteracciones[name]:
            for t in tiemposInteracciones[name][level]:
                if t["tiempo"] >= limiteOutliers[level]:
                    if level in outliersPorNivel[name]:
                        outliersPorNivel[name][level].append(t)
                    else:
                        outliersPorNivel[name][level] = [t]
                        
    return outliersPorNivel

In [ ]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None

In [ ]:
JSONFile = open('../data/trazasOrdenadas.json')
rawData = json.load(JSONFile)
JSONFile.close()

tenerEnCuentaMove = False
result = extraerTiemposEntreInteracciones(rawData, tenerEnCuentaMove)

resultados_Tiempos_Nivel_Jugador = extraerTiemposPorNivelJugador(rawData)

In [ ]:
tiemposPorNivel = tiempoPorNiveles_Jugador(resultados_Tiempos_Nivel_Jugador["tiempos"])
#pd.DataFrame(tiemposPorNivel)

In [ ]:
pd.DataFrame(resultados_Tiempos_Nivel_Jugador["intentosNecesarios"])

In [ ]:
intentosOrdenados = False
intentosMedios_CompletarNivel = getIntentosMedios_HastaCompletarNivel(resultados_Tiempos_Nivel_Jugador["intentosNecesarios"], intentosOrdenados)
print(intentosMedios_CompletarNivel["intentosMedios"])

In [ ]:
soloPrimerIntento = True
tiemposOrdenados = False
mediasNivel = getMediaTiempoPorNivel(tiemposPorNivel, soloPrimerIntento, tiemposOrdenados)
pd.DataFrame(mediasNivel["mediaTiempos"])

In [ ]:
#print(mediasNivel["mediaEstrellas"])

In [ ]:
ultNivelAlcanzado = getUltimoNivelAlcanzado(tiemposPorNivel)
pd.DataFrame.from_dict(ultNivelAlcanzado, orient='index').transpose()

In [ ]:
rankingNivelesNoEntendidos = getCategoriasPeorEntendidas(mediasNivel["mediaTiempos"], mediasNivel["mediaEstrellas"], intentosMedios_CompletarNivel, ultNivelAlcanzado, mediasNivel["listaNiveles"])
pd.DataFrame.from_dict(rankingNivelesNoEntendidos, orient='index').transpose()

In [ ]:
tiempos_Inicio_Interaccion = get_Tiempos_Ordenados(result["tiempoInicio_Interaccion_Jugador"])
tiempos_Interaccion_Completado = get_Tiempos_Ordenados(result["tiempoInteraccion_Completado_Jugador"])
tiempos_Interaccion_Interaccion = get_Tiempos_Ordenados(result["tiempoInteraccion_Interaccion_Jugador"])

outliers_Inicio_Interaccion = getLimiteOutliers(tiempos_Inicio_Interaccion)
outliers_Interaccion_Completado = getLimiteOutliers(tiempos_Interaccion_Completado)
outliers_Interaccion_Interaccion = getLimiteOutliers(tiempos_Interaccion_Interaccion)

In [ ]:
#Tiempo entre que inicia el nivel y realiza la primera interaccion, que interaccion realiza y si es completado si lo hace con exito
#Solo aparecen valores outliers
#Aqui apareceran jugadores que analizan el problema antes de ponerse a resolverlo
jugadoresOutliers_Inicio_Interaccion = getJugadoresOutliers(outliers_Inicio_Interaccion, result["tiempoInicio_Interaccion_Jugador"])
pd.DataFrame.from_dict(jugadoresOutliers_Inicio_Interaccion, orient='index').transpose()

In [ ]:
#Tiempo entre una interaccion (inicio, reinicio, interaccionTarjeta) y completar el nivel (con exito o salir al menu), que accion hizo antes
#Solo aparecen valores outliers
#Aqui apareceran jugadores que revisan si lo que han hecho tiene sentido y va a funcionar, "sin probar a lo loco"
jugadoresOutliers_Interaccion_Completado = getJugadoresOutliers(outliers_Inicio_Interaccion, result["tiempoInteraccion_Completado_Jugador"])
pd.DataFrame.from_dict(jugadoresOutliers_Interaccion_Completado, orient='index').transpose()

In [ ]:
#Tiempo entre una interaccion con tarjeta y otra interaccion con tarjeta, que accion hizo antes
#Solo aparecen valores outliers
#Aqui apareceran jugadores que piensan cada movimiento que hacen antes de hacerlo
jugadoresOutliers_Interaccion_Interaccion = getJugadoresOutliers(outliers_Inicio_Interaccion, result["tiempoInteraccion_Interaccion_Jugador"])
pd.DataFrame.from_dict(jugadoresOutliers_Interaccion_Interaccion, orient='index').transpose()

In [ ]:
pd.DataFrame(result["anomalias"])
#Se ha interactuado en un nivel que no estaba iniciado -> Probablemente sea porque completa el nivel, minimiza la ventana de siguiente nivel o menu y se pone a mover las tarjetas

In [ ]:
def verTrazas(rawData):
    for e in rawData:
        if("result" in e):
            if("extensions" in e["result"]):
                if("code" in e["result"]["extensions"]):
                    print(e["result"]["extensions"]["code"])
                    print("------------------------")
                    return ET.fromstring(e["result"]["extensions"]["code"])

JSONFile = open('trazasOrdenadas.json')
rawData = json.load(JSONFile)
parsed = verTrazas(rawData)
JSONFile.close()

antBloque = False

for child in parsed.iter():
    if child.tag == "block":
        if antBloque:
            print("Codigo muerto")
        else:
            antBloque = True
    else:
        antBloque = False

In [ ]:
#Podemos ordenar tiempos gracias a la sobrecarga de operadores:
tiempos = [Tiempo("1h/30m/15s"), Tiempo("1h/30m"), Tiempo("1h"), Tiempo("30m/17s"), Tiempo("30m"), Tiempo("28m"), Tiempo("1m/13s"), Tiempo("45s")]
tiempos.sort()
print(tiempos)

#Podemos hallar la media de una lista de tiempos
Tiempo(str(int(round(statistics.mean(map(int,tiempos)), 0))) + "s")

In [3]:
var = "\r\n<xml>\r\n    <variables />\r\n    <block type=\"start_start\" id=\"start_start_5pt5\" x=\"114\" y=\"-309\">\r\n        <next>\r\n            <block type=\"movement_move_laser\" id=\"movement_move_laser_8xbe\">\r\n                <field name=\"DIRECTION\">UP</field>\r\n                <next>\r\n                    <block type=\"movement_move_laser\" id=\"movement_move_laser_lanj\">\r\n                        <field name=\"DIRECTION\">RIGHT</field>\r\n                    </block>\r\n                </next>\r\n            </block>\r\n        </next>\r\n    </block>\r\n    <block type=\"movement_rotate_laser\" id=\"movement_rotate_laser_yc3j\" x=\"372\" y=\"-202\">\r\n        <field name=\"ROTATION\">CLOCKWISE</field>\r\n        <value name=\"AMOUNT\">\r\n            <block type=\"math_number\" id=\"math_number_uc66\">\r\n                <field name=\"NUM\">5</field>\r\n            </block>\r\n        </value>\r\n    </block>\r\n</xml>"

parsed = ET.fromstring(var)
print(var)
print("-------------------")
antBloque = False

for child in parsed:
    if child.tag == "block":
        if antBloque:
            print("Codigo muerto")
        else:
            antBloque = True
    else:
        antBloque = False


<xml>
    <variables />
    <block type="start_start" id="start_start_5pt5" x="114" y="-309">
        <next>
            <block type="movement_move_laser" id="movement_move_laser_8xbe">
                <field name="DIRECTION">UP</field>
                <next>
                    <block type="movement_move_laser" id="movement_move_laser_lanj">
                        <field name="DIRECTION">RIGHT</field>
                    </block>
                </next>
            </block>
        </next>
    </block>
    <block type="movement_rotate_laser" id="movement_rotate_laser_yc3j" x="372" y="-202">
        <field name="ROTATION">CLOCKWISE</field>
        <value name="AMOUNT">
            <block type="math_number" id="math_number_uc66">
                <field name="NUM">5</field>
            </block>
        </value>
    </block>
</xml>
-------------------
Codigo muerto


In [4]:
var = "\r\n<xml>\r\n    <variables>\r\n   <variable type=\"\" id=\"4Y1jFgEnfT+#?Z*jc9N_\">8Cantidad</variable>\r\n   <variable type=\"\" id=\"4Y1jFgEnfT+#?Z*jc9N_\">_prueba</variable>\r\n   </variables>\r\n  <block type=\"start_start\" id=\"start_start_5pt5\" x=\"114\" y=\"-309\">\r\n        <next>\r\n            <block type=\"movement_move_laser\" id=\"movement_move_laser_8xbe\">\r\n                <field name=\"DIRECTION\">UP</field>\r\n                <next>\r\n                    <block type=\"movement_move_laser\" id=\"movement_move_laser_lanj\">\r\n                        <field name=\"DIRECTION\">RIGHT</field>\r\n                    </block>\r\n                </next>\r\n            </block>\r\n        </next>\r\n    </block>\r\n    <block type=\"movement_rotate_laser\" id=\"movement_rotate_laser_yc3j\" x=\"372\" y=\"-202\">\r\n        <field name=\"ROTATION\">CLOCKWISE</field>\r\n        <value name=\"AMOUNT\">\r\n            <block type=\"math_number\" id=\"math_number_uc66\">\r\n                <field name=\"NUM\">5</field>\r\n            </block>\r\n        </value>\r\n    </block>\r\n</xml>"
print(var)
print("-----------------")
parsed = ET.fromstring(var)

pattern = re.compile(r'^_*[a-zA-Z][a-zA-Z0-9_]*$')

def nombreVarValido(nombresVar):
    declaracionVariables = []
    for nombre in nombresVar:
        if keyword.iskeyword(nombre):
            declaracionVariables.append(False)
        else:
            print(bool(pattern.match(nombre)))
            declaracionVariables.append(bool(pattern.match(nombre)))

    return declaracionVariables


variablesDeclaradas = []
for child in parsed.find("variables"):
    variablesDeclaradas.append(child.text)

print(variablesDeclaradas)
print(nombreVarValido(variablesDeclaradas))


<xml>
    <variables>
   <variable type="" id="4Y1jFgEnfT+#?Z*jc9N_">Cantidad</variable>
   <variable type="" id="4Y1jFgEnfT+#?Z*jc9N_">_prueba</variable>
   </variables>
  <block type="start_start" id="start_start_5pt5" x="114" y="-309">
        <next>
            <block type="movement_move_laser" id="movement_move_laser_8xbe">
                <field name="DIRECTION">UP</field>
                <next>
                    <block type="movement_move_laser" id="movement_move_laser_lanj">
                        <field name="DIRECTION">RIGHT</field>
                    </block>
                </next>
            </block>
        </next>
    </block>
    <block type="movement_rotate_laser" id="movement_rotate_laser_yc3j" x="372" y="-202">
        <field name="ROTATION">CLOCKWISE</field>
        <value name="AMOUNT">
            <block type="math_number" id="math_number_uc66">
                <field name="NUM">5</field>
            </block>
        </value>
    </block>
</xml>
------------

In [ ]:
def verTrazas(rawData):
    for e in rawData:
        if("auvqa" in e["actor"]["name"]):
            print(e)
            print("------------------------")

JSONFile = open('data/trazasOrdenadas.json')
rawData = json.load(JSONFile)
parsed = verTrazas(rawData)
JSONFile.close()

In [1]:
import os

#to get the current working directory
directory = os.getcwd()

print(directory)

c:\Users\Ángel Hortelano\Documents\Universidad\TFG\Github\TFG_Articoding_22-23\Back\OldCode
